# 개요

## 코스닥

In [55]:
import pandas as pd
import yfinance as yf

start = '2024-11-12'
end = '2024-11-23'

tickers = pd.read_csv(f'/Users/leesangwon/Downloads/코스닥이상치.csv', encoding="utf-8")
# 티커 칼럼 리스트화
tickers_list = tickers['Original_Ticker'].tolist() if 'Original_Ticker' in tickers.columns else tickers.iloc[:, 0].tolist()
symbol = tickers_list

stock = yf.download(symbol, start, end)
stock_kosdaq = stock['Adj Close']

# 날짜 인덱스를 칼럼으로 변환
stock_kosdaq_reset = stock_kosdaq.reset_index()

stock_kosdaq_reset

# CSV 파일로 저장
stock_kosdaq_reset.to_csv('/Users/leesangwon/Downloads/코스닥실제주가_1112_1119.csv', index=False)

[*********************100%***********************]  49 of 49 completed


## 코스피

In [56]:
import pandas as pd
import yfinance as yf

start = '2024-11-12'
end = '2024-11-20'

tickers = pd.read_csv(f'/Users/leesangwon/Downloads/코스피이상치.csv', encoding="cp949")
# 티커 칼럼 리스트화
tickers_list = tickers['Original Ticker'].tolist() if 'Original Ticker' in tickers.columns else tickers.iloc[:, 0].tolist()
symbol = tickers_list

stock = yf.download(symbol, start, end)
stock_kospi= stock['Adj Close']

# 날짜 인덱스를 칼럼으로 변환
stock_kospi_reset = stock_kospi.reset_index()

stock_kospi

# CSV 파일로 저장
stock_kospi_reset.to_csv('/Users/leesangwon/Downloads/코스피실제주가_1112_1119.csv', index=False)


[*********************100%***********************]  17 of 17 completed


## 예측주가

In [71]:
df = pd.read_csv(f'/Users/leesangwon/Downloads/코스닥이상치.csv', encoding="utf-8")


In [72]:
# Original_Ticker 칼럼의 고유값을 리스트로 변환
original_tickers = df['Original_Ticker'].tolist()

# 결과 출력
original_tickers

['307930.KS',
 '053280.KS',
 '036000.KS',
 '226950.KS',
 '418470.KS',
 '291650.KS',
 '127120.KS',
 '099440.KS',
 '298380.KS',
 '255220.KS',
 '039560.KS',
 '115610.KS',
 '445680.KS',
 '424960.KS',
 '088800.KS',
 '220100.KS',
 '207760.KS',
 '067310.KS',
 '403870.KS',
 '226400.KS',
 '021080.KS',
 '115180.KS',
 '039020.KS',
 '050110.KS',
 '038460.KS',
 '019570.KS',
 '027830.KS',
 '317770.KS',
 '109670.KS',
 '043610.KS',
 '317530.KS',
 '256630.KS',
 '290090.KS',
 '028080.KS',
 '079940.KS',
 '314130.KS',
 '319400.KS',
 '053450.KS',
 '293780.KS',
 '090710.KS',
 '070300.KS',
 '039310.KS',
 '304840.KS',
 '205100.KS',
 '041440.KS',
 '399720.KS',
 '149950.KS',
 '057030.KS',
 '315640.KS']

In [73]:
# 필요한 라이브러리 로드
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# 티커 리스트
symbols = original_tickers


# 예측 결과를 담을 DataFrame
forecast_results = pd.DataFrame()

# 티커별로 예측 수행
for symbol in symbols:
    # 1. 데이터 다운로드 (수정 종가 사용)
    data = yf.download(symbol, start="2024-04-10", end="2024-10-11")
    data = data['Adj Close']
    
    if data.empty:
        print(f"데이터 다운로드 실패: {symbol}")
        continue
    else:
        print(f"{symbol} 데이터: {data.head()}")
    
    # 2. 데이터 전처리
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    # 3. 학습 데이터 생성
    sequence_length = 60
    X_train, y_train = [], []
    for i in range(sequence_length, len(scaled_data)):
        X_train.append(scaled_data[i-sequence_length:i, 0])
        y_train.append(scaled_data[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # 4. LSTM 모델 생성
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 5. 모델 학습
    model.fit(X_train, y_train, batch_size=32, epochs=20)

    # 6. 예측 준비
    test_data = scaled_data[-sequence_length:]
    test_data = np.array([test_data[:, 0]])
    test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], 1))

    # 7. 29일 예측
    predictions = []
    for _ in range(29):
        next_pred = model.predict(test_data)[0][0]
        predictions.append(next_pred)
        test_data = np.append(test_data[:, 1:, :], [[[next_pred]]], axis=1)

    # 8. 예측 결과 복원
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    # 9. 예측 결과 시각화
    forecast_index = pd.date_range(start=data.index[-1], periods=30, freq='B')[1:]
    
    # 예측 결과를 DataFrame에 저장
    forecast_df = pd.DataFrame({'Date': forecast_index, f'{symbol} Forecasted': predictions.flatten()})
    forecast_df.set_index('Date', inplace=True)

    # 결과를 종합 DataFrame에 추가
    if forecast_results.empty:
        forecast_results = forecast_df
    else:
        forecast_results = pd.merge(forecast_results, forecast_df, left_index=True, right_index=True, how='outer')

# 예측 결과 출력
forecast_results

[*********************100%***********************]  1 of 1 completed

307930.KS 데이터: Date
2024-04-11    6630.0
2024-04-12    6760.0
2024-04-15    6790.0
2024-04-16    6400.0
2024-04-17    6340.0
Name: Adj Close, dtype: float64
Epoch 1/20



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.1548
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0696
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0244
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0099
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0227
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0208
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0124
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0094
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0102
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0119
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0126
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0111
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0088
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0103
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0096
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/

[*********************100%***********************]  1 of 1 completed

...

[*********************100%***********************]  1 of 1 completed

315640.KS 데이터: Date
2024-04-11    7390.0
2024-04-12    7330.0
2024-04-15    7150.0
2024-04-16    7180.0
2024-04-17    7100.0
Name: Adj Close, dtype: float64
Epoch 1/20



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0990 
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0331
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0128
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0197
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0205
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0116
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0098
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0110
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0134
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0130
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0119
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0118
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0105
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0093
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0095
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms

,307930.KS Forecasted,053280.KS Forecasted,036000.KS Forecasted,226950.KS Forecasted,418470.KS Forecasted,291650.KS Forecasted,127120.KS Forecasted,099440.KS Forecasted,298380.KS Forecasted,255220.KS Forecasted,...,090710.KS Forecasted,070300.KS Forecasted,039310.KS Forecasted,304840.KS Forecasted,205100.KS Forecasted,041440.KS Forecasted,399720.KS Forecasted,149950.KS Forecasted,057030.KS Forecasted,315640.KS Forecasted
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-11,5140.425781,5195.014648,1692.894409,18035.947266,15243.872070,2311.161621,3038.150146,3115.373291,31206.197266,2361.607178,...,1257.878662,5659.582031,1575.535522,3545.797852,1781.246094,7708.178711,41987.246094,11790.892578,3554.066895,5515.477539
2024-10-14,5140.786621,5185.875488,1705.398438,18155.113281,15247.987305,2329.274902,3044.000488,3116.318359,31313.490234,2368.861816,...,1250.322632,5782.211914,1576.299072,3543.191650,1779.807251,7711.741699,41947.066406,11764.536133,3554.196777,5512.711426
2024-10-15,5140.766602,5179.432129,1713.788330,18202.968750,15256.041016,2342.543457,3049.901611,3116.452881,31362.408203,2378.814697,...,1240.922607,5946.937012,1576.445923,3542.307129,1778.300171,7720.374023,41884.054688,11739.068359,3553.251221,5508.215820
2024-10-16,5140.516602,5174.920898,1719.306152,18207.044922,15266.112305,2352.765625,3055.630859,3116.097412,31374.285156,2390.458252,...,1230.383301,6140.121582,1576.159912,3542.677979,1776.770508,7731.491699,41808.007812,11714.916016,3551.698242,5502.700195
2024-10-17,5140.133301,5171.769043,1722.820312,18184.826172,15277.039062,2361.031494,3061.089844,3115.459473,31363.033203,2403.100586,...,1219.195923,6351.801270,1575.582397,3543.928955,1775.246704,7743.534668,41725.097656,11692.333008,3549.822510,5496.653809
2024-10-18,5139.679688,5169.554199,1724.940308,18147.511719,15288.127930,2368.002930,3066.240234,3114.671631,31337.705078,2416.256104,...,1207.695801,6574.388184,1574.817505,3545.767822,1773.745728,7755.589844,41639.148438,11671.451172,3547.797363,5490.407227
2024-10-21,5139.193848,5167.972656,1726.093140,18102.193359,15298.985352,2374.083740,3071.074219,3113.818359,31304.167969,2429.578613,...,1196.107788,6802.102539,1573.939453,3547.974121,1772.277588,7767.152832,41552.527344,11652.317383,3545.726562,5484.172852
2024-10-22,5138.699707,5166.810547,1726.579468,18053.283203,15309.392578,2379.521240,3075.599854,3112.950928,31266.193359,2442.813477,...,1184.581177,7030.416992,1573.000000,3550.386719,1770.848511,7777.964844,41466.644531,11634.925781,3543.671875,5478.086426
2024-10-23,5138.212402,5165.921875,1726.610229,18003.462891,15319.242188,2384.469238,3079.830811,3112.102051,31226.183594,2455.770020,...,1173.210815,7255.671387,1572.033936,3552.891113,1769.461182,7787.920898,41382.332031,11619.231445,3541.670166,5472.228027


In [74]:
# 날짜 인덱스를 칼럼으로 변환
forecast_results_reset = forecast_results.reset_index()

forecast_results_reset

forecast_results_reset.to_csv('/Users/leesangwon/Downloads/코스닥예측주가_1011_1120.csv', index=False)
